In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import glob

In [2]:
# Machine-learning specific imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## API Imports 
import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

start_date = '2022-01-01'
end_date = '2022-01-31'

## FMP Functions 
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    
    ## Export and save data so API calls aren't repeated 
    path = Path('../FilesExport_Updated_API_data/'+symbol+'_jan_2022.pkl')
    save_obj(df,path)
    
    return df

In [6]:
def get_data(symbol, n_days):
    path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
    data = load_obj(path)
    df = data[symbol]
    
    df_close = df[['close']]
    df_close = df_close.reset_index().rename(columns={"Date": "Close_Date"})
    
    features_df = df.reset_index().drop(columns=['close','adjClose'])
    
    new_close_df = df_close.iloc[n_days: , :].reset_index(drop=True)
    
    ## Prevent multiple API calls each time, but use API when needed. 
    try:
        path = Path('../FilesExport_Updated_API_data/'+symbol+'_jan_2022.pkl')
        api_df = load_obj(path)
    except:
        api_df = get_FMP_historical_data(symbol)
    

    new_data = api_df[['close']]
    new_data = new_data.reset_index().rename(columns={"Date": "Close_Date"})
    new_data = new_data.iloc[0:n_days]
    
    new_close_df = new_close_df.append(new_data, ignore_index=True)
    

    
    return features_df, new_close_df

## Dropping QUANTITY_FAILS from dataframe before machine learning 

def prepare_data(symbol,n_days,return_data=False):
    features_df, new_close_df = get_data(symbol,n_days)
    
    X = features_df.drop(columns={'Date','QUANTITY_FAILS'}).values
    y = new_close_df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    
    if return_data == True:
        return X, y, features_df, new_close_df
    else:
        return X, y

In [7]:
## Takes one symbol and runs model data. 
## Call function for each individal symbol. No return data.  

def mean_squared_model(
    symbol,
    export_path,  ## Require export path to help avoiding re-writing models 
    n_days = 5, ## Default value 5, but should be tried with 1-30  
    model_count = 5, ## Number of times model runs before saving the best one. 
    validation_split_value=0.3, ## Default 0.3 for 70/30 split 
    epochs_value=400,
    units1 = 8,
    units2 = 8,
    num_of_inputs = 20,
    num_of_outputs= 1,
    model_type = 'NN'
    ):    

    X, y, = prepare_data(symbol,n_days)
    
    n_days_string = str(n_days) ## For exporting because can't concat 'int'
    accuracy_dict_symbol = str(symbol)+'_'+n_days_string
    export_path_prefix_lowAcc = export_path+'Low_Acc/'+symbol+'_'+model_type+'_'+n_days_string 
    export_path_prefix_highAcc = export_path+'High_Acc/'+symbol+'_'+model_type+'_'+n_days_string 


    for i in range(model_count):
        model_summary = {}
        ## Create Neural Network 

        # Define the model - deep neural network with two layers
        nn = Sequential()

        # First hidden layer
        nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn.add(Dense(units=units2, activation="relu"))

        # Output layer
        nn.add(Dense(units=num_of_outputs, activation="linear"))

        # Compile the model
        nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model = nn.fit(X, y, 
                          validation_split=validation_split_value, 
                          epochs=epochs_value, 
                          verbose=0)
            
        ## End of if/elif

        model_loss, model_accuracy = nn.evaluate(X, y, verbose=0)
        
        model_summary = {
            'model_accuracy':model_accuracy,
            'n_days':n_days,
            'validation_split_value':validation_split_value,
            'epochs_value':epochs_value,
                        }
        
        ## Save first model 
        if i == 0:
            model_accuracy_ = model_accuracy
            symbol_accuracy_dict[accuracy_dict_symbol] = {
                'model_accuracy':model_accuracy,
                'n_days':n_days
            }
            
            # Save model data Low_Acc
            nn_json = nn.to_json()
            file_path = Path(export_path_prefix_lowAcc+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = (export_path_prefix_lowAcc+'_model_weights.h5')
            nn.save_weights(file_path)
            
            ## Save model summary 
            file_path = (export_path_prefix_lowAcc+'_model_summary.pkl')
            save_obj(model_summary,file_path)
            
            # Save model data High_Acc
            nn_json = nn.to_json()
            file_path = Path(export_path_prefix_highAcc+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = (export_path_prefix_highAcc+'_model_weights.h5')
            nn.save_weights(file_path)
            
            ## Save model summary 
            file_path = (export_path_prefix_highAcc+'_model_summary.pkl')
            save_obj(model_summary,file_path)
            
        ## Rewrite saved model if accuracy better 
        else:
            if model_accuracy < model_accuracy_:
                ## Rewrite values 
                model_accuracy_ = model_accuracy
                symbol_accuracy_dict[accuracy_dict_symbol] = {
                    'model_accuracy':model_accuracy,
                    'n_days':n_days
                }
                
                # Rewrite saved files
                
                ## Save model data
                nn_json = nn.to_json()
                file_path = Path(export_path_prefix_lowAcc+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = (export_path_prefix_lowAcc+'_model_weights.h5')
                nn.save_weights(file_path)
                
                ## Save model summary 
                file_path = (export_path_prefix_lowAcc+'_model_summary.pkl')
                save_obj(model_summary,file_path)
                
            if model_accuracy > model_accuracy_:
                ## Rewrite values 
                model_accuracy_ = model_accuracy
                symbol_accuracy_dict[accuracy_dict_symbol] = {
                    'model_accuracy':model_accuracy,
                    'n_days':n_days
                }
                
                # Rewrite saved files
                
                ## Save model data
                nn_json = nn.to_json()
                file_path = Path(export_path_prefix_highAcc+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = (export_path_prefix_highAcc+'_model_weights.h5')
                nn.save_weights(file_path)
                
                ## Save model summary 
                file_path = (export_path_prefix_highAcc+'_model_summary.pkl')
                save_obj(model_summary,file_path)
    ## End of for loop 
    ## Return nothing 

In [8]:
## Import symbol list 
path = Path('../Resources/06_01_ML_symbol_success_list.pkl')
symbol_list = load_obj(path)
len(symbol_list)

770

In [9]:
## Create empty dict for accuracy data - only really needed when 
## looping multiple symbols or different models. 
symbol_accuracy_dict = {}

for symbol in symbol_list:
    i = 1
    mean_squared_model(symbol,
                       export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                       n_days=i,
                       epochs_value=200,
                       validation_split_value=0.1,
                       num_of_inputs=19
                      )
    i = 2
    mean_squared_model(symbol,
                       export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                       n_days=i,
                       epochs_value=200,
                       validation_split_value=0.1,
                       num_of_inputs=19
                      )
    i = 5
    mean_squared_model(symbol,
                       export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                       n_days=i,
                       epochs_value=200,
                       validation_split_value=0.1,
                       num_of_inputs=19
                      )
    i = 10
    mean_squared_model(symbol,
                       export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                       n_days=i,
                       epochs_value=200,
                       validation_split_value=0.1,
                       num_of_inputs=19
                      )

## Export symbol_accuracy_dict
path = Path('../Resources/NN_noFTDall_symbol_list.pkl')
save_obj(symbol_accuracy_dict,path)

KeyboardInterrupt: 

In [ ]:
symbol_accuracy_dict 

In [ ]:
# ## load model data 
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_NN_model_data.json')
# with open(file_path, "r") as json_file:
#     model_json = json_file.read()
# loaded_model = model_from_json(model_json)

# # load weights into new model
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_NN_model_weights.h5')
# loaded_model.load_weights(file_path)

# # Load model summary
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_model_summary.pkl')
# model_summary = load_obj(file_path)

# X , y, features, close_df = prepare_data(symbol,n_days,return_data=True)


# close_df["predicted"] = loaded_model.predict(X)

In [ ]:
# export_path='../Model_Data/Date_Test_NN/'
# model_type = 'NN'

# model_summary_list = [] 

# for i in range(1,11):
#     # Load model summary
#     n_days_string = str(i)
#     file_path = Path('_model_summary.pkl')
#     model_summary = load_obj(file_path)
    
#     model_summary_list.append(model_summary)
# model_summary_list

In [ ]:
# n_days = 1
# model_type = 'NN'

# export_path='../Model_Data/Date_Test_NN_noFTD/'
# export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string
# n_days_string = str(n_days)

# file_path = Path(export_path_prefix+'_model_data.json')
# with open(file_path, "r") as json_file:
#     model_json = json_file.read()
# loaded_model = model_from_json(model_json)

# # load weights into new model
# file_path = Path(export_path_prefix+'_model_weights.h5')
# loaded_model.load_weights(file_path)

# X , y, features, close_df = prepare_data(symbol,n_days,return_data=True)


# close_df["predicted"] = loaded_model.predict(X)
# close_df.set_index('Close_Date',inplace=True)
# close_df[['close','predicted']].tail(30).plot(use_index=True)